In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms

from torchvision import datasets
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
### TODO: Write data loaders for training, validation, and test sets
## Specify appropriate transforms, and batch_sizes
data_dir = './dogImages'

train_transforms = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_data = datasets.ImageFolder(data_dir+'/train', transform=train_transforms)
valid_data = datasets.ImageFolder(data_dir+'/valid', transform=test_transforms)
test_data  = datasets.ImageFolder(data_dir+'/test',  transform=test_transforms)

n_classes = len(train_data.classes)

batch_size = 20
# Setting shuffle=True can improve the performance a lot!
loaders_transfer = {
    'train': torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True),
    'valid': torch.utils.data.DataLoader(valid_data, batch_size=batch_size, shuffle=True),
    'test':  torch.utils.data.DataLoader(test_data,  batch_size=batch_size, shuffle=True)
}
      
#Check if gpu support is available
use_cuda = torch.cuda.is_available()

In [3]:
model_transfer = models.vgg16(pretrained=True)

# freeze training for all "features" layers
for param in model_transfer.features.parameters():
    param.requires_grad = False

n_in = model_transfer.classifier[6].in_features
model_transfer.classifier[6] = nn.Linear(in_features=n_in, out_features=n_classes)

if use_cuda:
    model_transfer.cuda()

print(model_transfer)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [4]:
### TODO: select loss function
criterion_transfer = nn.CrossEntropyLoss()

# specify optimizer
optimizer_transfer = optim.SGD(model_transfer.classifier.parameters(), lr=1e-3)

In [5]:
def train(n_epochs, loaders, model, optimizer, criterion, use_cuda, save_path):
    """returns trained model"""
    # initialize tracker for minimum validation loss
    valid_loss_min = np.Inf 
    
    for epoch in range(1, n_epochs+1):
        # initialize variables to monitor training and validation loss
        train_loss = 0.0
        valid_loss = 0.0
        
        ###################
        # train the model #
        ###################
        model.train()
        for batch_idx, (data, target) in enumerate(loaders['train']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## find the loss and update the model parameters accordingly
            ## record the average training loss, using something like
            ## train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        ######################    
        # validate the model #
        ######################
        model.eval()
        for batch_idx, (data, target) in enumerate(loaders['valid']):
            # move to GPU
            if use_cuda:
                data, target = data.cuda(), target.cuda()
            ## update the average validation loss
            output = model(data)
            loss = criterion(output, target)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.data - valid_loss))
        
        # print training/validation statistics 
        print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
        
        ## TODO: save the model if validation loss has decreased
        if valid_loss < valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
                valid_loss_min,
                valid_loss))
            torch.save(model.state_dict(), save_path)
            valid_loss_min = valid_loss
            
    # return trained model
    return model

In [6]:
%%time
# train the model
n_epochs = 10
model_transfer = train(n_epochs, loaders_transfer, model_transfer, optimizer_transfer,
                       criterion_transfer, use_cuda, 'model_transfer.pt')

Epoch: 1 	Training Loss: 4.258833 	Validation Loss: 2.876933
Validation loss decreased (inf --> 2.876933).  Saving model ...
Epoch: 2 	Training Loss: 2.704753 	Validation Loss: 1.287238
Validation loss decreased (2.876933 --> 1.287238).  Saving model ...
Epoch: 3 	Training Loss: 1.813054 	Validation Loss: 0.774625
Validation loss decreased (1.287238 --> 0.774625).  Saving model ...
Epoch: 4 	Training Loss: 1.499444 	Validation Loss: 0.617342
Validation loss decreased (0.774625 --> 0.617342).  Saving model ...
Epoch: 5 	Training Loss: 1.323535 	Validation Loss: 0.527340
Validation loss decreased (0.617342 --> 0.527340).  Saving model ...
Epoch: 6 	Training Loss: 1.234531 	Validation Loss: 0.480409
Validation loss decreased (0.527340 --> 0.480409).  Saving model ...
Epoch: 7 	Training Loss: 1.177452 	Validation Loss: 0.462266
Validation loss decreased (0.480409 --> 0.462266).  Saving model ...
Epoch: 8 	Training Loss: 1.084772 	Validation Loss: 0.443145
Validation loss decreased (0.46226

In [7]:
def test(loaders, model, criterion, use_cuda):

    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.

    model.eval()
    for batch_idx, (data, target) in enumerate(loaders['test']):
        # move to GPU
        if use_cuda:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update average test loss 
        test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
        # convert output probabilities to predicted class
        pred = output.data.max(1, keepdim=True)[1]
        # compare predictions to true label
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print('Test Loss: {:.6f}\n'.format(test_loss))

    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

# call test function    
test(loaders_transfer, model_transfer, criterion_transfer, use_cuda)

Test Loss: 0.459532


Test Accuracy: 85% (714/836)


In [8]:
### TODO: Write a function that takes a path to an image as input
### and returns the dog breed that is predicted by the model.

# list of class names by index, i.e. a name can be accessed like class_names[0]
class_names = [item[4:].replace("_", " ") for item in train_data.classes]

def predict_breed_transfer(img_path, model, use_cuda):
    # load the image and return the predicted breed
    img = Image.open(img_path).convert('RGB')
    in_transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    img = in_transform(img)[:3,:,:].unsqueeze(0)
    if use_cuda:
        img = img.cuda()
    model.eval()
    output = model(img)
    idx = output.argmax().item()
    return class_names[idx]

In [9]:
img_path = './images/Brittany_02625.jpg'
predict_breed_transfer(img_path, model_transfer, use_cuda)

'Brittany'